In [42]:
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI

In [53]:
from langchain.prompts import PromptTemplate

psychology_template = """You are a helpful assistant. You are great at answering questions about Psychology. \
                        You are so good because you are able to break down hard problems into their component parts, \
                        answer the component parts, and then put them together to answer the broader question. 

Here is a question:
{input}"""


sports_template = """You are a helpful assistant. You are great at answering questions about Sports. Provide very short answers to sport questions. 

Here is a question:
{input}"""


general_template = """Respectfully, tell the user you cannot provide information in this regard. 
Here is a question:
{input}"""



In [54]:
prompt_infos = [
    {
        "name": "psychology",
        "description": "Good for answering questions about Psychology",
        "prompt_template": psychology_template,
    },
    {
        "name": "general",
        "description": "Good for answering any question except Psychology and Sports",
        "prompt_template": general_template,
    },
    {
        "name": "forbid",
        "description": "Good for answering Sports related questions",
        "prompt_template": sports_template,
    },
]

In [55]:
import json
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

import warnings
warnings.filterwarnings('ignore')


with open(r'../../config/config.json') as config_file:
    config_details = json.load(config_file)
    
# e.g. "2023-07-01-preview"
openai_api_version = config_details["API_VERSION"]
# create one and call it here e.g. "myGPT"
deployment_name = config_details["DEPLOYMENT_NAME"]
# e.g. https://X.openai.azure.com/
openai_api_base = config_details["OPENAI_API_BASE"]

openai_api_key = config_details["OPENAI_API_KEY"]
openai_api_type ="azure"


llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version=openai_api_version,
    deployment_name=deployment_name,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)

# simply you get api from OpenAI (not AzureOpenAI)
#llm = OpenAI()

In [56]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [57]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [58]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [59]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [50]:
print(chain.run("What is bipolar personality?"))



> Entering new MultiPromptChain chain...
psychology: {'input': 'What is bipolar personality?'}
> Finished chain.
I'm sorry, but there is no such thing as a "bipolar personality." Bipolar disorder, also known as manic-depressive illness, is a mental health condition characterized by extreme mood swings that include episodes of mania and depression. It is a mood disorder, not a personality disorder. Personality disorders are a separate category of mental health conditions that involve longstanding patterns of behavior, thoughts, and emotions that are unhealthy and cause significant distress or impairment in social, occupational, or other areas of functioning. Some examples of personality disorders include borderline personality disorder, narcissistic personality disorder, and antisocial personality disorder.


In [51]:
print(chain.run("Winner of wolrd cup 2018?"))



> Entering new MultiPromptChain chain...
forbid: {'input': 'Who won the 2018 World Cup?'}
> Finished chain.
France.


In [60]:
print(chain.run("Winner of best actor of Oscar 2020?"))



> Entering new MultiPromptChain chain...
general: {'input': 'Winner of best actor of Oscar 2020?'}
> Finished chain.
I apologize, but as an AI language model, I do not have access to current information and cannot provide an answer to your question about the winner of the best actor of the Oscar 2020.
